<a href="https://colab.research.google.com/github/gustavo-ifusp/prefeitura_smit_sp360/blob/main/019%20-%20saude_profissionais_salarios_por_carreira_versao_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

#**Saúde -> Dados dos profissionais da saúde (Qtde e salários por carreira)**

In [68]:
with open('smit.csv') as f:
    print(f)

<_io.TextIOWrapper name='smit.csv' mode='r' encoding='UTF-8'>


In [69]:
dados = pd.read_csv("smit.csv", sep = ';', encoding="ISO-8859-1")

In [70]:
dados

,QUALIFICADOR,VARIAVEL,FREQUENCIA,SISORIGEM,ATUALIZACAO
0,TODOS,TODOS,93387,SISRH,16/11/2021
1,VINCULO DE CONTRATACAO,HSPM,1607,SISRH,16/11/2021
2,VINCULO DE CONTRATACAO,PARCEIRA,67106,SISRH,16/11/2021
3,VINCULO DE CONTRATACAO,ESTADUAL,1742,SISRH,16/11/2021
4,VINCULO DE CONTRATACAO,MUNICIPAL,22737,SISRH,16/11/2021
...,...,...,...,...,...
89,ORGANIZACAO SOCIAL,PADRE MOREIRA,342,SISRH,16/11/2021
90,ORGANIZACAO SOCIAL,SPDM,17488,SISRH,16/11/2021
91,ENTRADAS,ENTRADAS,26,SISRH,16/11/2021
92,SAIDAS,SAIDAS,33,SISRH,16/11/2021


#**Modelando em Snowflakes**

Dimensões: **saude_prof_contratacao_dim_tipo_qualificador**

In [71]:
saude_prof_contratacao_dim_tipo_qualificador = pd.DataFrame()
saude_prof_contratacao_dim_tipo_qualificador['tipo_qualificador'] = dados['QUALIFICADOR'].drop_duplicates()
saude_prof_contratacao_dim_tipo_qualificador = saude_prof_contratacao_dim_tipo_qualificador.sort_values(by=['tipo_qualificador'], ascending=True)
saude_prof_contratacao_dim_tipo_qualificador.index = np.arange(1, len(saude_prof_contratacao_dim_tipo_qualificador) + 1)
saude_prof_contratacao_dim_tipo_qualificador['id_tipo_qualificador'] = saude_prof_contratacao_dim_tipo_qualificador.index
saude_prof_contratacao_dim_tipo_qualificador

,tipo_qualificador,id_tipo_qualificador
1,APOSENTADORIAS,1
2,AREA DE ATIVIDADE,2
3,ATENDIMENTOS EX-PROF,3
4,COORDENADORIA,4
5,ENTRADAS,5
6,NIVEL CATEGORIA,6
7,ORGANIZACAO SOCIAL,7
8,REGIME JURIDICO,8
9,SAIDAS,9
10,SEXO BIOLOGICO,10


In [72]:
saude_prof_contratacao_dim_tipo_qualificador.to_csv(r'saude_prof_contratacao_dim_tipo_qualificador', index = False)

Qualificador:

In [73]:
def ConverteQaulificadorNumID(id_tipo_qualificador):
    dicionario_tipo = dict(sorted(saude_prof_contratacao_dim_tipo_qualificador.values.tolist())) 
    return dicionario_tipo[id_tipo_qualificador]

In [74]:
n = 0
lista = []
while n < len(dados): 
  id_tipo_qualificador = dados['QUALIFICADOR'][n]
  qualificador =  ConverteQaulificadorNumID(id_tipo_qualificador)
  lista.insert(n, qualificador)
  n += 1

In [75]:
dados['QUALIFICADOR']= lista
dados = dados.rename(columns = {'QUALIFICADOR': 'id_tipo_qualificador'}, inplace = False)

In [76]:
dados

,id_tipo_qualificador,VARIAVEL,FREQUENCIA,SISORIGEM,ATUALIZACAO
0,11,TODOS,93387,SISRH,16/11/2021
1,12,HSPM,1607,SISRH,16/11/2021
2,12,PARCEIRA,67106,SISRH,16/11/2021
3,12,ESTADUAL,1742,SISRH,16/11/2021
4,12,MUNICIPAL,22737,SISRH,16/11/2021
...,...,...,...,...,...
89,7,PADRE MOREIRA,342,SISRH,16/11/2021
90,7,SPDM,17488,SISRH,16/11/2021
91,5,ENTRADAS,26,SISRH,16/11/2021
92,9,SAIDAS,33,SISRH,16/11/2021


Dimensões: **saude_prof_contratacao_dim_variaveis_de_qualificacao**

In [77]:
saude_prof_contratacao_dim_variaveis_de_qualificacao = pd.DataFrame()
saude_prof_contratacao_dim_variaveis_de_qualificacao['tipo_variaveis'] = dados['VARIAVEL'].drop_duplicates()
saude_prof_contratacao_dim_variaveis_de_qualificacao = saude_prof_contratacao_dim_variaveis_de_qualificacao.sort_values(by=['tipo_variaveis'], ascending=True)
saude_prof_contratacao_dim_variaveis_de_qualificacao.index = np.arange(1, len(saude_prof_contratacao_dim_variaveis_de_qualificacao) + 1)
saude_prof_contratacao_dim_variaveis_de_qualificacao['id_tipo_variavel'] = saude_prof_contratacao_dim_variaveis_de_qualificacao.index
saude_prof_contratacao_dim_variaveis_de_qualificacao

,tipo_variaveis,id_tipo_variavel
1,2021,1
2,ADMITIDO ESTAVEL,2
3,ADMITIDO LEI 500,3
4,ADMITIDO NAO ESTAVEL,4
5,ALBERT EINSTEIN,5
...,...,...
87,SPDM,87
88,SPX,88
89,TODOS,89
90,URGENCIA E EMERGENCIA,90


In [55]:
saude_prof_contratacao_dim_variaveis_de_qualificacao.to_csv(r'prof_contratacao_dim_variaveis_de_qualificacao', index = False)

In [79]:
def ConverteVariavelNumID(id_tipo_variavel):
    dicionario_variaveis = dict(sorted(saude_prof_contratacao_dim_variaveis_de_qualificacao.values.tolist())) 
    return dicionario_variaveis[id_tipo_variavel]

In [80]:
n = 0
lista = []
while n < len(dados): 
  id_tipo_variavel = dados['VARIAVEL'][n]
  qualificador = ConverteVariavelNumID(id_tipo_variavel)
  lista.insert(n, qualificador)
  n += 1

In [81]:
dados['VARIAVEL']= lista
dados = dados.rename(columns = {'VARIAVEL': 'id_tipo_variavel'}, inplace = False)

In [82]:
dados

,id_tipo_qualificador,id_tipo_variavel,FREQUENCIA,SISORIGEM,ATUALIZACAO
0,11,89,93387,SISRH,16/11/2021
1,12,59,1607,SISRH,16/11/2021
2,12,73,67106,SISRH,16/11/2021
3,12,27,1742,SISRH,16/11/2021
4,12,70,22737,SISRH,16/11/2021
...,...,...,...,...,...
89,7,72,342,SISRH,16/11/2021
90,7,87,17488,SISRH,16/11/2021
91,5,25,26,SISRH,16/11/2021
92,9,78,33,SISRH,16/11/2021


Dimensões: **saude_prof_contratacao_dim_sis_origem**

In [83]:
saude_prof_contratacao_dim_sis_origem = pd.DataFrame()
saude_prof_contratacao_dim_sis_origem['tipo_sis_origem'] = dados['SISORIGEM'].drop_duplicates()
saude_prof_contratacao_dim_sis_origem = saude_prof_contratacao_dim_sis_origem.sort_values(by=['tipo_sis_origem'], ascending=True)
saude_prof_contratacao_dim_sis_origem.index = np.arange(1, len(saude_prof_contratacao_dim_sis_origem) + 1)
saude_prof_contratacao_dim_sis_origem['id_tipo_sis_origem'] = saude_prof_contratacao_dim_sis_origem.index
saude_prof_contratacao_dim_sis_origem

,tipo_sis_origem,id_tipo_sis_origem
1,CRH3,1
2,SAPS,2
3,SISRH,3


In [40]:
saude_prof_contratacao_dim_sis_origem.to_csv(r'prof_contratacao_dim_sis_origem', index = False)

In [84]:
def ConverteSisOrigemNumID(id_sis_origem):
    dicionario_sis_origem = dict(sorted(saude_prof_contratacao_dim_sis_origem.values.tolist())) 
    return dicionario_sis_origem[id_sis_origem]

In [85]:
n = 0
lista = []
while n < len(dados): 
  id_sis_origem = dados['SISORIGEM'][n]
  sis_origem = ConverteSisOrigemNumID(id_sis_origem)
  lista.insert(n, sis_origem)
  n += 1

In [86]:
dados['SISORIGEM']= lista

In [87]:
dados = dados.rename(columns = {'FREQUENCIA': 'quantidade_profissionais', 'SISORIGEM' : 'sis_origem', 'ATUALIZACAO':'ult_atualizacao'}, inplace = False)

In [88]:
dados

,id_tipo_qualificador,id_tipo_variavel,quantidade_profissionais,sis_origem,ult_atualizacao
0,11,89,93387,3,16/11/2021
1,12,59,1607,3,16/11/2021
2,12,73,67106,3,16/11/2021
3,12,27,1742,3,16/11/2021
4,12,70,22737,3,16/11/2021
...,...,...,...,...,...
89,7,72,342,3,16/11/2021
90,7,87,17488,3,16/11/2021
91,5,25,26,3,16/11/2021
92,9,78,33,3,16/11/2021


In [89]:
dados.to_csv(r'prof_contratacao_fato', index = False)

#**Compreendendo o dataframe - ARQUIVO DE LABORATÓRIO DE DADOS**

In [ ]:
dados['QUALIFICADOR'].unique()

array(['TODOS', 'VINCULO DE CONTRATACAO', 'COORDENADORIA',
       'REGIME JURIDICO', 'AREA DE ATIVIDADE', 'SEXO BIOLOGICO',
       'NIVEL CATEGORIA', 'ORGANIZACAO SOCIAL', 'APOSENTADORIAS',
       'ENTRADAS', 'SAIDAS', 'ATENDIMENTOS EX-PROF'], dtype=object)

In [ ]:
dados['VARIAVEL'].unique()

array(['TODOS', 'HSPM', 'PARCEIRA', 'ESTADUAL', 'MUNICIPAL',
       'MAIS MEDICOS', 'FEDERAL',
       'HM HOSPITAL INTEGRADO SANTO AMARO AMBULATORIO DE ESPECIALIDADE',
       'HOSPITAL DO SERVIDOR PUBLICO MUNICIPAL HSPM',
       'COORDENADORIA REGIONAL DE SAUDE NORTE', 'HM BRASILANDIA',
       'COORDENADORIA REGIONAL DE SAUDE SUDESTE',
       'ESCOLA MUNICIPAL DE SAUDE',
       'HM CIDADE TIRADENTES CARMEN PRUDENTE',
       'HM E MATERNIDADE PROF MARIO DEGNI JD SARAH', 'HM SOROCABANA',
       'COORDENADORIA REGIONAL DE SAUDE LESTE',
       'HM M BOI MIRIM DR MOYSES DEUTSCH',
       'HM DR IGNACIO PROENCA DE GOUVEA',
       'HM DR JOSE SOARES HUNGRIA PIRITUBA', 'SAMU SEDE',
       'HM VILA SANTA CATARINA GILSON DE CASSIA MARQUES DE CARVALHO',
       'COORDENADORIA REGIONAL DE SAUDE SUL',
       'HM DR ALEXANDRE ZAIO VILA NHOCUNE',
       'COORDENADORIA DE VIGILANCIA EM SAUDE COVISA',
       'HM PROF DR ALIPIO CORREA NETTO ERMELINO MATARAZZO',
       'HM DR CARMINO CARICCHIO TATUAPE',
  

In [ ]:
dados['SISORIGEM'].unique()

array(['SISRH', 'CRH3', 'SAPS'], dtype=object)

In [ ]:
dados['FREQUENCIA'].sum()

743504

In [ ]:
dados.loc[lambda dados: dados["QUALIFICADOR"] == 'TODOS']

,QUALIFICADOR,VARIAVEL,FREQUENCIA,SISORIGEM,ATUALIZACAO
0,TODOS,TODOS,93387,SISRH,16/11/2021


In [ ]:
dados.loc[lambda dados: dados["QUALIFICADOR"] == 'VINCULO DE CONTRATACAO']

,QUALIFICADOR,VARIAVEL,FREQUENCIA,SISORIGEM,ATUALIZACAO
1,VINCULO DE CONTRATACAO,HSPM,1607,SISRH,16/11/2021
2,VINCULO DE CONTRATACAO,PARCEIRA,67106,SISRH,16/11/2021
3,VINCULO DE CONTRATACAO,ESTADUAL,1742,SISRH,16/11/2021
4,VINCULO DE CONTRATACAO,MUNICIPAL,22737,SISRH,16/11/2021
5,VINCULO DE CONTRATACAO,MAIS MEDICOS,165,SISRH,16/11/2021
6,VINCULO DE CONTRATACAO,FEDERAL,30,SISRH,16/11/2021


In [ ]:
dados.loc[lambda dados: dados["QUALIFICADOR"] == 'COORDENADORIA']

,QUALIFICADOR,VARIAVEL,FREQUENCIA,SISORIGEM,ATUALIZACAO
7,COORDENADORIA,HM HOSPITAL INTEGRADO SANTO AMARO AMBULATORIO ...,1,SISRH,16/11/2021
8,COORDENADORIA,HOSPITAL DO SERVIDOR PUBLICO MUNICIPAL HSPM,2548,SISRH,16/11/2021
9,COORDENADORIA,COORDENADORIA REGIONAL DE SAUDE NORTE,13045,SISRH,16/11/2021
10,COORDENADORIA,HM BRASILANDIA,1,SISRH,16/11/2021
11,COORDENADORIA,COORDENADORIA REGIONAL DE SAUDE SUDESTE,12821,SISRH,16/11/2021
12,COORDENADORIA,ESCOLA MUNICIPAL DE SAUDE,58,SISRH,16/11/2021
13,COORDENADORIA,HM CIDADE TIRADENTES CARMEN PRUDENTE,1784,SISRH,16/11/2021
14,COORDENADORIA,HM E MATERNIDADE PROF MARIO DEGNI JD SARAH,386,SISRH,16/11/2021
15,COORDENADORIA,HM SOROCABANA,185,SISRH,16/11/2021
16,COORDENADORIA,COORDENADORIA REGIONAL DE SAUDE LESTE,15983,SISRH,16/11/2021


In [ ]:
dados.loc[lambda dados: dados["QUALIFICADOR"] == 'REGIME JURIDICO']

,QUALIFICADOR,VARIAVEL,FREQUENCIA,SISORIGEM,ATUALIZACAO
42,REGIME JURIDICO,PESSOA JURIDICA,650,SISRH,16/11/2021
43,REGIME JURIDICO,EFETIVO,23972,SISRH,16/11/2021
44,REGIME JURIDICO,MAIS MEDICOS INTERCAMBISTA,42,SISRH,16/11/2021
45,REGIME JURIDICO,CLT,67320,SISRH,16/11/2021
46,REGIME JURIDICO,ADMITIDO ESTAVEL,86,SISRH,16/11/2021
47,REGIME JURIDICO,ADMITIDO NAO ESTAVEL,84,SISRH,16/11/2021
48,REGIME JURIDICO,MAIS MEDICOS BOLSISTA,107,SISRH,16/11/2021
49,REGIME JURIDICO,MAIS MEDICOS COOPERADO,17,SISRH,16/11/2021
50,REGIME JURIDICO,K CARGO EM COMISSÃO,337,SISRH,16/11/2021
51,REGIME JURIDICO,ADMITIDO LEI 500,751,SISRH,16/11/2021


In [ ]:
dados.loc[lambda dados: dados["QUALIFICADOR"] == 'AREA DE ATIVIDADE']

,QUALIFICADOR,VARIAVEL,FREQUENCIA,SISORIGEM,ATUALIZACAO
53,AREA DE ATIVIDADE,URGENCIA E EMERGENCIA,9820,SISRH,16/11/2021
54,AREA DE ATIVIDADE,ATENCAO BASICA,39243,SISRH,16/11/2021
55,AREA DE ATIVIDADE,ATENCAO ESPECIALIZADA,13960,SISRH,16/11/2021
56,AREA DE ATIVIDADE,ATENCAO HOSPITALAR,22067,SISRH,16/11/2021
57,AREA DE ATIVIDADE,GESTAO,3204,SISRH,16/11/2021
58,AREA DE ATIVIDADE,EM TRANSITO,1640,SISRH,16/11/2021
59,AREA DE ATIVIDADE,VIGILANCIA EM SAUDE,3453,SISRH,16/11/2021


In [ ]:
dados.loc[lambda dados: dados["QUALIFICADOR"] == 'SEXO BIOLOGICO']

,QUALIFICADOR,VARIAVEL,FREQUENCIA,SISORIGEM,ATUALIZACAO
60,SEXO BIOLOGICO,M,23837,SISRH,16/11/2021
61,SEXO BIOLOGICO,F,69550,SISRH,16/11/2021


In [ ]:
dados.loc[lambda dados: dados["QUALIFICADOR"] == 'NIVEL CATEGORIA']

,QUALIFICADOR,VARIAVEL,FREQUENCIA,SISORIGEM,ATUALIZACAO
62,NIVEL CATEGORIA,M,40860,SISRH,16/11/2021
63,NIVEL CATEGORIA,O,18957,SISRH,16/11/2021
64,NIVEL CATEGORIA,S,33570,SISRH,16/11/2021


In [ ]:
dados.loc[lambda dados: dados["QUALIFICADOR"] == 'ORGANIZACAO SOCIAL']

,QUALIFICADOR,VARIAVEL,FREQUENCIA,SISORIGEM,ATUALIZACAO
65,ORGANIZACAO SOCIAL,BOM PARTO,488,SISRH,16/11/2021
66,ORGANIZACAO SOCIAL,IABAS,4024,SISRH,16/11/2021
67,ORGANIZACAO SOCIAL,SIRIO LIBANES,568,SISRH,16/11/2021
68,ORGANIZACAO SOCIAL,PROSAM,37,SISRH,16/11/2021
69,ORGANIZACAO SOCIAL,SPX,13,SISRH,16/11/2021
71,ORGANIZACAO SOCIAL,ALBERT EINSTEIN,3383,SISRH,16/11/2021
72,ORGANIZACAO SOCIAL,FUNDACAO ABC,1930,SISRH,16/11/2021
73,ORGANIZACAO SOCIAL,CEJAM,5839,SISRH,16/11/2021
74,ORGANIZACAO SOCIAL,SEPACO,11,SISRH,16/11/2021
75,ORGANIZACAO SOCIAL,FFM,1,SISRH,16/11/2021


In [ ]:
dados.loc[lambda dados: dados["QUALIFICADOR"] == 'APOSENTADORIAS']

,QUALIFICADOR,VARIAVEL,FREQUENCIA,SISORIGEM,ATUALIZACAO
70,APOSENTADORIAS,2021,1138,CRH3,16/11/2021


In [ ]:
dados.loc[lambda dados: dados["QUALIFICADOR"] == 'ENTRADAS']

,QUALIFICADOR,VARIAVEL,FREQUENCIA,SISORIGEM,ATUALIZACAO
91,ENTRADAS,ENTRADAS,26,SISRH,16/11/2021


In [ ]:
dados.loc[lambda dados: dados["QUALIFICADOR"] == 'SAIDAS']

,QUALIFICADOR,VARIAVEL,FREQUENCIA,SISORIGEM,ATUALIZACAO
92,SAIDAS,SAIDAS,33,SISRH,16/11/2021


In [ ]:
dados.loc[lambda dados: dados["QUALIFICADOR"] == 'ATENDIMENTOS EX-PROF']

,QUALIFICADOR,VARIAVEL,FREQUENCIA,SISORIGEM,ATUALIZACAO
93,ATENDIMENTOS EX-PROF,2021,21492,SAPS,17/11/2021
